In [1]:
import os
from getpass import getpass

user = getpass("Github User:")
password = getpass("Github Password:")
os.environ['GITHUB_AUTH'] = user + ':' + password

Github User:··········
Github Password:··········


In [2]:
REPO_DIR = '/demo'
%mkdir -p "$REPO_DIR"
%cd "$REPO_DIR" 
!git clone https://$GITHUB_AUTH@github.com/srijarkoroy/LinkSeg 
%cd "$REPO_DIR/LinkSeg"

/demo
Cloning into 'LinkSeg'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 17), reused 34 (delta 7), pack-reused 0
Unpacking objects: 100% (49/49), 10.91 KiB | 1.21 MiB/s, done.
/demo/LinkSeg


In [3]:
!unzip /content/drive/MyDrive/datasets.zip -d /demo/LinkSeg

Archive:  /content/drive/MyDrive/datasets.zip
mapname:  conversion of  failed
 extracting: /demo/LinkSeg/test.zip  
 extracting: /demo/LinkSeg/training.zip  


In [4]:
%mkdir dataset/

In [5]:
!unzip /demo/LinkSeg/test.zip -d /demo/LinkSeg/dataset/

Archive:  /demo/LinkSeg/test.zip
   creating: /demo/LinkSeg/dataset/test/
   creating: /demo/LinkSeg/dataset/test/images/
  inflating: /demo/LinkSeg/dataset/test/images/01_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/02_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/03_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/04_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/05_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/06_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/07_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/08_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/09_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/10_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/11_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/12_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/13_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/14_test.tif  
  inflating: /demo/LinkS

In [6]:
!unzip /demo/LinkSeg/training.zip -d /demo/LinkSeg/dataset/

Archive:  /demo/LinkSeg/training.zip
   creating: /demo/LinkSeg/dataset/training/
   creating: /demo/LinkSeg/dataset/training/1st_manual/
 extracting: /demo/LinkSeg/dataset/training/1st_manual/21_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/22_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/23_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/24_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/25_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/26_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/27_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/28_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/29_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/30_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/31_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/32_manual1.gif 

In [7]:
data_path = "/demo/LinkSeg/dataset"
print(data_path)

/demo/LinkSeg/dataset


In [8]:
!cat data/augment.py

import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm

import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate


def create_dir(path):

    ### Create a directory
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):

    ### Split data into train and test set
    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "mask", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

def augment_data(images, masks, save_path, augment=True):

    ### Augmenting data into Horizontal,Vertical & Rotate

    size = (512, 512)

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the name """
        name = x.split("/")[-1].split(".")[0]

 

In [9]:
from data.augment import load_data
(train_x, train_y), (test_x, test_y) = load_data(data_path)

In [10]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 20 - 20
Test: 20 - 20


In [11]:
from data.augment import create_dir

create_dir("/demo/LinkSeg/new_data/train/image")
create_dir("/demo/LinkSeg/new_data/train/mask")
create_dir("/demo/LinkSeg/new_data/test/image/")
create_dir("/demo/LinkSeg/new_data/test/mask/")

In [12]:
from data.augment import augment_data

augment_data(train_x, train_y, "/demo/LinkSeg/new_data/train", augment=True)
augment_data(test_x, test_y, "/demo/LinkSeg/new_data/test", augment=False)

100%|██████████| 20/20 [00:00<00:00, 26.74it/s]
